In [1]:
import pymongo
cli = pymongo.MongoClient()
db = cli.test
coll = db.restaurants

In [2]:
coll.drop_indexes()

In [3]:
q = coll.find({'name': 'Dunkin Donuts'})
plan = q.explain()
plan

{'executionStats': {'allPlansExecution': [],
  'executionStages': {'advanced': 200,
   'direction': 'forward',
   'docsExamined': 25359,
   'executionTimeMillisEstimate': 11,
   'filter': {'name': {'$eq': 'Dunkin Donuts'}},
   'invalidates': 0,
   'isEOF': 1,
   'nReturned': 200,
   'needTime': 25160,
   'needYield': 0,
   'restoreState': 198,
   'saveState': 198,
   'stage': 'COLLSCAN',
   'works': 25361},
  'executionSuccess': True,
  'executionTimeMillis': 17,
  'nReturned': 200,
  'totalDocsExamined': 25359,
  'totalKeysExamined': 0},
 'ok': 1.0,
 'queryPlanner': {'indexFilterSet': False,
  'namespace': 'test.restaurants',
  'parsedQuery': {'name': {'$eq': 'Dunkin Donuts'}},
  'plannerVersion': 1,
  'rejectedPlans': [],
  'winningPlan': {'direction': 'forward',
   'filter': {'name': {'$eq': 'Dunkin Donuts'}},
   'stage': 'COLLSCAN'}},
 'serverInfo': {'gitVersion': '3f76e40c105fc223b3e5aac3e20dcd026b83b38b',
  'host': 'mongo-cur',
  'port': 27017,
  'version': '3.4.2'}}

In [4]:
plan.keys()

dict_keys(['queryPlanner', 'executionStats', 'serverInfo', 'ok'])

In [5]:
plan['queryPlanner']

{'indexFilterSet': False,
 'namespace': 'test.restaurants',
 'parsedQuery': {'name': {'$eq': 'Dunkin Donuts'}},
 'plannerVersion': 1,
 'rejectedPlans': [],
 'winningPlan': {'direction': 'forward',
  'filter': {'name': {'$eq': 'Dunkin Donuts'}},
  'stage': 'COLLSCAN'}}

In [6]:
plan['executionStats']

{'allPlansExecution': [],
 'executionStages': {'advanced': 200,
  'direction': 'forward',
  'docsExamined': 25359,
  'executionTimeMillisEstimate': 11,
  'filter': {'name': {'$eq': 'Dunkin Donuts'}},
  'invalidates': 0,
  'isEOF': 1,
  'nReturned': 200,
  'needTime': 25160,
  'needYield': 0,
  'restoreState': 198,
  'saveState': 198,
  'stage': 'COLLSCAN',
  'works': 25361},
 'executionSuccess': True,
 'executionTimeMillis': 17,
 'nReturned': 200,
 'totalDocsExamined': 25359,
 'totalKeysExamined': 0}

In [7]:
from pprint import pprint
from copy import deepcopy

def plan_summary(plan, print_winning_plan=False):
    plan = deepcopy(plan)
    stages = []
    stage = plan['executionStats']['executionStages']
    while stage:
        inputStage = stage.pop('inputStage', None)
        stages.append(stage)
        stage = inputStage
    stages.reverse()
    if print_winning_plan:
        print('Winning Plan')
        winningPlan = plan['queryPlanner']['winningPlan']
        pprint(winningPlan)
        print()
    print('Execution Stages')
    for stage in stages:
        pprint(stage)
        print('--')

In [8]:
plan_summary(plan, print_winning_plan=True)

Winning Plan
{'direction': 'forward',
 'filter': {'name': {'$eq': 'Dunkin Donuts'}},
 'stage': 'COLLSCAN'}

Execution Stages
{'advanced': 200,
 'direction': 'forward',
 'docsExamined': 25359,
 'executionTimeMillisEstimate': 11,
 'filter': {'name': {'$eq': 'Dunkin Donuts'}},
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 200,
 'needTime': 25160,
 'needYield': 0,
 'restoreState': 198,
 'saveState': 198,
 'stage': 'COLLSCAN',
 'works': 25361}
--


In [9]:
coll.create_index('name')

'name_1'

In [10]:
q = coll.find({'name': 'Dunkin Donuts'})
plan = q.explain()
plan_summary(plan, print_winning_plan=True)

Winning Plan
{'inputStage': {'direction': 'forward',
                'indexBounds': {'name': ['["Dunkin Donuts", "Dunkin Donuts"]']},
                'indexName': 'name_1',
                'indexVersion': 1,
                'isMultiKey': False,
                'isPartial': False,
                'isSparse': False,
                'isUnique': False,
                'keyPattern': {'name': 1},
                'multiKeyPaths': {'name': []},
                'stage': 'IXSCAN'},
 'stage': 'FETCH'}

Execution Stages
{'advanced': 200,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'name': ['["Dunkin Donuts", "Dunkin Donuts"]']},
 'indexName': 'name_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'name': 1},
 'keysExamined': 200,
 'multiKeyPaths': {'name': []},
 'nReturned': 200,
 'needTime': 0,
 'needYield': 0,
 'restoreSta

In [11]:
q = coll.find({'name': 'Dunkin Donuts', 'cuisine': 'Armenian'})
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 200,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'name': ['["Dunkin Donuts", "Dunkin Donuts"]']},
 'indexName': 'name_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'name': 1},
 'keysExamined': 200,
 'multiKeyPaths': {'name': []},
 'nReturned': 200,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 1,
 'saveState': 1,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 201}
--
{'advanced': 1,
 'alreadyHasObj': 0,
 'docsExamined': 200,
 'executionTimeMillisEstimate': 0,
 'filter': {'cuisine': {'$eq': 'Armenian'}},
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 1,
 'needTime': 199,
 'needYield': 0,
 'restoreState': 1,
 'saveState': 1,
 'stage': 'FETCH',
 'works': 201}
--


In [12]:
coll.create_index([('name', 1), ('cuisine', 1)])
coll.drop_index('name_1')

In [13]:
q = coll.find({'name': 'Dunkin Donuts', 'cuisine': 'Armenian'})
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 1,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'cuisine': ['["Armenian", "Armenian"]'],
                 'name': ['["Dunkin Donuts", "Dunkin Donuts"]']},
 'indexName': 'name_1_cuisine_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'cuisine': 1, 'name': 1},
 'keysExamined': 1,
 'multiKeyPaths': {'cuisine': [], 'name': []},
 'nReturned': 1,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 2}
--
{'advanced': 1,
 'alreadyHasObj': 0,
 'docsExamined': 1,
 'executionTimeMillisEstimate': 0,
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 1,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'stage': 'FETCH',
 'works': 2}
--


## Indexes and `$or`

In [14]:
coll.drop_indexes()
coll.create_index('name')
coll.create_index('address.zipcode')

'address.zipcode_1'

In [15]:
q = coll.find({'$or': [
    {'name': 'Dunkin Donuts'}, 
    {'address.zipcode': '10462'}]})
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 350,
 'dupsDropped': 0,
 'dupsTested': 350,
 'executionTimeMillisEstimate': 0,
 'inputStages': [{'advanced': 200,
                  'direction': 'forward',
                  'dupsDropped': 0,
                  'dupsTested': 0,
                  'executionTimeMillisEstimate': 0,
                  'indexBounds': {'name': ['["Dunkin Donuts", "Dunkin '
                                           'Donuts"]']},
                  'indexName': 'name_1',
                  'indexVersion': 1,
                  'invalidates': 0,
                  'isEOF': 1,
                  'isMultiKey': False,
                  'isPartial': False,
                  'isSparse': False,
                  'isUnique': False,
                  'keyPattern': {'name': 1},
                  'keysExamined': 200,
                  'multiKeyPaths': {'name': []},
                  'nReturned': 200,
                  'needTime': 0,
                  'needYield': 0,
                  'restoreState

In [16]:
def print_stage(stage, indent=0, keys=(
        'stage', 'direction', 'keysExamined', 'nReturned',
        'parsedQuery', 'parsedTextQuery')):
    inputs = stage.pop('inputStages', [])
    if 'inputStage' in stage:
        inputs.append(stage.pop('inputStage'))
    toprint = {k: stage.get(k, None) for k in keys}
    toprint = {k: v for k, v in toprint.items() if v is not None}
    print('   ' * indent, toprint)
    for istage in inputs:
        print_stage(istage, indent + 1)

def short_plan_summary(plan, plan_number=None):
    stages = []
    if plan_number is None:
        stage = plan['executionStats']['executionStages']
    else:
        stage = plan['executionStats']['allPlansExecution'][plan_number]['executionStages']
    print_stage(deepcopy(stage))

In [17]:
short_plan_summary(plan)

 {'stage': 'SUBPLAN', 'nReturned': 350}
    {'stage': 'FETCH', 'nReturned': 350}
       {'stage': 'OR', 'nReturned': 350}
          {'stage': 'IXSCAN', 'direction': 'forward', 'keysExamined': 200, 'nReturned': 200}
          {'stage': 'IXSCAN', 'direction': 'forward', 'keysExamined': 150, 'nReturned': 150}


## Using `$and` (real and implied)

In [18]:
q = coll.find({'$and': [
    {'name': 'Dunkin Donuts'}, 
    {'address.zipcode': '10462'}]})
plan = q.explain()
short_plan_summary(plan)

 {'stage': 'FETCH', 'nReturned': 0}
    {'stage': 'IXSCAN', 'direction': 'forward', 'keysExamined': 150, 'nReturned': 150}


In [19]:
q = coll.find({'$and': [
    {'name': 'Dunkin Donuts'}, 
    {'address.zipcode': '10462'}]})
plan = q.explain()
short_plan_summary(plan, plan_number=1)

 {'stage': 'FETCH', 'nReturned': 0}
    {'stage': 'AND_SORTED', 'nReturned': 0}
       {'stage': 'IXSCAN', 'direction': 'forward', 'keysExamined': 59, 'nReturned': 59}
       {'stage': 'IXSCAN', 'direction': 'forward', 'keysExamined': 92, 'nReturned': 92}


## Range / prefix queries

In [20]:
import re
q = coll.find({'name': re.compile('^Dunkin.* Donuts$')})
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 408,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 19,
 'filter': {'name': {'$options': 'u', '$regex': '^Dunkin.* Donuts$'}},
 'indexBounds': {'name': ['["", {})',
                          '[/^Dunkin.* Donuts$/u, /^Dunkin.* Donuts$/u]']},
 'indexName': 'name_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'name': 1},
 'keysExamined': 25359,
 'multiKeyPaths': {'name': []},
 'nReturned': 408,
 'needTime': 24951,
 'needYield': 0,
 'restoreState': 198,
 'saveState': 198,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 25360}
--
{'advanced': 408,
 'alreadyHasObj': 0,
 'docsExamined': 408,
 'executionTimeMillisEstimate': 19,
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 408,
 'needTime': 24951,
 'needYield': 0,
 'restoreState': 198,
 'saveState': 198,
 'stage': 'FETCH',
 'works': 25360}
--


In [21]:
q.distinct('name')

['Dunkin  Donuts', 'Dunkin Donuts', "Dunkin'  Donuts", "Dunkin' Donuts"]

In [22]:
coll.drop_indexes()
coll.create_index([('cuisine', 1), ('name', 1)])

'cuisine_1_name_1'

In [23]:
q = coll.find({'name': re.compile('^Dunkin.* Donuts$')})
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 408,
 'direction': 'forward',
 'docsExamined': 25359,
 'executionTimeMillisEstimate': 22,
 'filter': {'name': {'$options': 'u', '$regex': '^Dunkin.* Donuts$'}},
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 408,
 'needTime': 24952,
 'needYield': 0,
 'restoreState': 198,
 'saveState': 198,
 'stage': 'COLLSCAN',
 'works': 25361}
--


In [24]:
q = coll.find({'name': re.compile('^Dunkin.* Donuts$'), 'cuisine': 'Donuts'})
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 344,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'filter': {'name': {'$options': 'u', '$regex': '^Dunkin.* Donuts$'}},
 'indexBounds': {'cuisine': ['["Donuts", "Donuts"]'],
                 'name': ['["", {})',
                          '[/^Dunkin.* Donuts$/u, /^Dunkin.* Donuts$/u]']},
 'indexName': 'cuisine_1_name_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'cuisine': 1, 'name': 1},
 'keysExamined': 480,
 'multiKeyPaths': {'cuisine': [], 'name': []},
 'nReturned': 344,
 'needTime': 135,
 'needYield': 0,
 'restoreState': 3,
 'saveState': 3,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 480}
--
{'advanced': 344,
 'alreadyHasObj': 0,
 'docsExamined': 344,
 'executionTimeMillisEstimate': 0,
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 344,
 'needTime': 135,
 'needYield': 0,
 '

In [25]:
coll.find_one()

{'_id': ObjectId('58dae60d8b8e4054555b72f8'),
 'address': {'building': '1007',
  'coord': [-73.856077, 40.848447],
  'street': 'Morris Park Ave',
  'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
   'grade': 'A',
   'score': 2},
  {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6},
  {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10},
  {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2011, 3, 10, 0, 0), 'grade': 'B', 'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}

In [26]:
coll.drop_indexes()
coll.create_index('name')

'name_1'

In [27]:
q = coll.find({
    'name': 'Dunkin Donuts', 
    'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 200,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'name': ['["Dunkin Donuts", "Dunkin Donuts"]']},
 'indexName': 'name_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'name': 1},
 'keysExamined': 200,
 'multiKeyPaths': {'name': []},
 'nReturned': 200,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 1,
 'saveState': 1,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 201}
--
{'advanced': 9,
 'alreadyHasObj': 0,
 'docsExamined': 200,
 'executionTimeMillisEstimate': 0,
 'filter': {'$and': [{'address.zipcode': {'$lte': '10462'}},
                     {'address.zipcode': {'$gte': '10460'}}]},
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 9,
 'needTime': 191,
 'needYield': 0,
 'restoreState': 1,
 'saveState': 1,
 'stage': 'FETCH',
 'works': 201}
--


In [28]:
coll.create_index([('name',1), ('address.zipcode', 1)])

'name_1_address.zipcode_1'

In [29]:
q = coll.find({'name': 'Dunkin Donuts', 'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 9,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'address.zipcode': ['["10460", "10462"]'],
                 'name': ['["Dunkin Donuts", "Dunkin Donuts"]']},
 'indexName': 'name_1_address.zipcode_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'address.zipcode': 1, 'name': 1},
 'keysExamined': 9,
 'multiKeyPaths': {'address.zipcode': [], 'name': []},
 'nReturned': 9,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 10}
--
{'advanced': 9,
 'alreadyHasObj': 0,
 'docsExamined': 9,
 'executionTimeMillisEstimate': 0,
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 9,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'stage': 'FETCH',
 'works': 11}
--


## Sorting 

In [30]:
coll.drop_indexes()
coll.create_index('name')
q = coll.find({'name': 'Dunkin Donuts'})
q = q.sort('address.zipcode')
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 200,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'name': ['["Dunkin Donuts", "Dunkin Donuts"]']},
 'indexName': 'name_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'name': 1},
 'keysExamined': 200,
 'multiKeyPaths': {'name': []},
 'nReturned': 200,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 3,
 'saveState': 3,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 201}
--
{'advanced': 200,
 'alreadyHasObj': 0,
 'docsExamined': 200,
 'executionTimeMillisEstimate': 0,
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 200,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 3,
 'saveState': 3,
 'stage': 'FETCH',
 'works': 201}
--
{'advanced': 200,
 'executionTimeMillisEstimate': 0,
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 200,
 'needTime': 1,
 'needYield': 0,
 'resto

In [31]:
coll.drop_indexes()
coll.create_index([('name', 1), ('address.zipcode', 1)])

'name_1_address.zipcode_1'

In [32]:
q = coll.find({'name': 'Dunkin Donuts'})
q = q.sort('address.zipcode')
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 200,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'address.zipcode': ['[MinKey, MaxKey]'],
                 'name': ['["Dunkin Donuts", "Dunkin Donuts"]']},
 'indexName': 'name_1_address.zipcode_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'address.zipcode': 1, 'name': 1},
 'keysExamined': 200,
 'multiKeyPaths': {'address.zipcode': [], 'name': []},
 'nReturned': 200,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 1,
 'saveState': 1,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 201}
--
{'advanced': 200,
 'alreadyHasObj': 0,
 'docsExamined': 200,
 'executionTimeMillisEstimate': 0,
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 200,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 1,
 'saveState': 1,
 'stage': 'FETCH',
 'works': 201}
--


In [33]:
q = coll.find({
    'name': 'Dunkin Donuts', 
    'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
q = q.sort('borough')
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 9,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'address.zipcode': ['["10460", "10462"]'],
                 'name': ['["Dunkin Donuts", "Dunkin Donuts"]']},
 'indexName': 'name_1_address.zipcode_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'address.zipcode': 1, 'name': 1},
 'keysExamined': 9,
 'multiKeyPaths': {'address.zipcode': [], 'name': []},
 'nReturned': 9,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 10}
--
{'advanced': 9,
 'alreadyHasObj': 0,
 'docsExamined': 9,
 'executionTimeMillisEstimate': 0,
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 9,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'stage': 'FETCH',
 'works': 10}
--
{'advanced': 9,
 'executionTimeMill

In [34]:
coll.drop_indexes()
coll.create_index([
    ('name', 1),
    ('borough', 1),
    ('address.zipcode', 1)
])
q = coll.find({
    'name': 'Dunkin Donuts', 
    'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
q = q.sort('borough')
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 9,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'address.zipcode': ['["10460", "10462"]'],
                 'borough': ['[MinKey, MaxKey]'],
                 'name': ['["Dunkin Donuts", "Dunkin Donuts"]']},
 'indexName': 'name_1_borough_1_address.zipcode_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 1,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'address.zipcode': 1, 'borough': 1, 'name': 1},
 'keysExamined': 16,
 'multiKeyPaths': {'address.zipcode': [], 'borough': [], 'name': []},
 'nReturned': 9,
 'needTime': 6,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'seeks': 7,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 16}
--
{'advanced': 9,
 'alreadyHasObj': 0,
 'docsExamined': 9,
 'executionTimeMillisEstimate': 0,
 'invalidates': 0,
 'isEOF': 1,
 'nReturned': 9,
 'needTime': 6,
 'needYield': 0,
 'restoreState': 0

# `explain()` geospatial queries

In [35]:
doc = coll.find_one()
doc

{'_id': ObjectId('58dae60d8b8e4054555b72f8'),
 'address': {'building': '1007',
  'coord': [-73.856077, 40.848447],
  'street': 'Morris Park Ave',
  'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
   'grade': 'A',
   'score': 2},
  {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6},
  {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10},
  {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2011, 3, 10, 0, 0), 'grade': 'B', 'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}

In [36]:
coll.create_index([
    ('address.coord', '2dsphere')  # longitude, latitude 
])

'address.coord_2dsphere'

In [37]:
q = coll.find(
    {'address.coord': {
        '$nearSphere': doc['address']['coord']
    }},
    {'_id': 0, 'name': 1, 'address.coord': 1}
).limit(5)
plan = q.explain()
plan_summary(plan)

Execution Stages
{'advanced': 5,
 'executionTimeMillisEstimate': 0,
 'indexName': 'address.coord_2dsphere',
 'indexVersion': 1,
 'inputStages': [{'advanced': 1,
                  'alreadyHasObj': 0,
                  'docsExamined': 1,
                  'executionTimeMillisEstimate': 0,
                  'inputStage': {'advanced': 1,
                                 'direction': 'forward',
                                 'dupsDropped': 0,
                                 'dupsTested': 0,
                                 'executionTimeMillisEstimate': 0,
                                 'indexBounds': {'address.coord': ['[-8574853690513424384, '
                                                                   '-8574853690513424384]',
                                                                   '[-8520810494984978432, '
                                                                   '-8520810494984978432]',
                                                                   '[

In [38]:
def print_stage(stage, indent=0, keys=(
        'stage', 'direction', 'keysExamined', 'nReturned',
        'parsedQuery', 'parsedTextQuery')):
    inputs = stage.pop('inputStages', [])
    if 'inputStage' in stage:
        inputs.append(stage.pop('inputStage'))
    toprint = {k: stage.get(k, None) for k in keys}
    toprint = {k: v for k, v in toprint.items() if v is not None}
    print('   ' * indent, toprint)
    for istage in inputs:
        print_stage(istage, indent + 1)

def short_plan_summary(plan):
    stages = []
    stage = plan['executionStats']['executionStages']
    print_stage(deepcopy(stage))

In [39]:
plan = q.explain()
short_plan_summary(plan)

 {'stage': 'LIMIT', 'nReturned': 5}
    {'stage': 'PROJECTION', 'nReturned': 5}
       {'stage': 'GEO_NEAR_2DSPHERE', 'nReturned': 5}
          {'stage': 'FETCH', 'nReturned': 1}
             {'stage': 'IXSCAN', 'direction': 'forward', 'keysExamined': 15, 'nReturned': 1}
          {'stage': 'FETCH', 'nReturned': 1}
             {'stage': 'IXSCAN', 'direction': 'forward', 'keysExamined': 15, 'nReturned': 1}
          {'stage': 'FETCH', 'nReturned': 0}
             {'stage': 'IXSCAN', 'direction': 'forward', 'keysExamined': 14, 'nReturned': 0}
          {'stage': 'FETCH', 'nReturned': 1}
             {'stage': 'IXSCAN', 'direction': 'forward', 'keysExamined': 16, 'nReturned': 1}
          {'stage': 'FETCH', 'nReturned': 4}
             {'stage': 'IXSCAN', 'direction': 'forward', 'keysExamined': 22, 'nReturned': 4}


# `explain()` full-text queries

In [40]:
coll.create_index([('name', 'text')])

'name_text'

In [41]:
q_text = coll.find(
    {'$text': {
        '$search': 'Dunkin Donuts',
        '$caseSensitive': True
    }})
plan = q_text.explain()
short_plan_summary(plan)

 {'stage': 'TEXT', 'nReturned': 588, 'parsedTextQuery': {'terms': ['Donut', 'Dunkin'], 'negatedTerms': [], 'phrases': [], 'negatedPhrases': []}}
    {'stage': 'TEXT_MATCH', 'nReturned': 588}
       {'stage': 'TEXT_OR', 'nReturned': 588}
          {'stage': 'IXSCAN', 'direction': 'backward', 'keysExamined': 587, 'nReturned': 587}
          {'stage': 'IXSCAN', 'direction': 'backward', 'keysExamined': 553, 'nReturned': 553}


In [42]:
q_text = coll.find(
    {'$text': {
        '$search': 'Dunkin Donuts',
        '$caseSensitive': True
    }},
    {'score': {'$meta': 'textScore'}, 'name': 1, '_id': 0})
q_text = q_text.sort(
    [('score', {'$meta': 'textScore'})])
plan = q_text.explain()
short_plan_summary(plan)

 {'stage': 'PROJECTION', 'nReturned': 588}
    {'stage': 'SORT', 'nReturned': 588}
       {'stage': 'SORT_KEY_GENERATOR', 'nReturned': 588}
          {'stage': 'TEXT', 'nReturned': 588, 'parsedTextQuery': {'terms': ['Donut', 'Dunkin'], 'negatedTerms': [], 'phrases': [], 'negatedPhrases': []}}
             {'stage': 'TEXT_MATCH', 'nReturned': 588}
                {'stage': 'TEXT_OR', 'nReturned': 588}
                   {'stage': 'IXSCAN', 'direction': 'backward', 'keysExamined': 587, 'nReturned': 587}
                   {'stage': 'IXSCAN', 'direction': 'backward', 'keysExamined': 553, 'nReturned': 553}


In [43]:
q_text = coll.find(
    {'$text': {
        '$search': '"Dunkin Donuts"',
        '$caseSensitive': True
    }})
plan = q_text.explain()
short_plan_summary(plan)

 {'stage': 'TEXT', 'nReturned': 257, 'parsedTextQuery': {'terms': ['Donut', 'Dunkin'], 'negatedTerms': [], 'phrases': ['Dunkin Donuts'], 'negatedPhrases': []}}
    {'stage': 'TEXT_MATCH', 'nReturned': 257}
       {'stage': 'TEXT_OR', 'nReturned': 588}
          {'stage': 'IXSCAN', 'direction': 'backward', 'keysExamined': 587, 'nReturned': 587}
          {'stage': 'IXSCAN', 'direction': 'backward', 'keysExamined': 553, 'nReturned': 553}
